In [8]:
import pandas as pd
import numpy as np

In [9]:
df = pd.read_csv('/home/sjkim/recommendSystem/finalproject/change_df.csv',index_col=0)
df = df[['user_id','item_id','votes','timestamp','rating','compound_score']]
df.head()

,user_id,item_id,votes,timestamp,rating,compound_score
0,B00191WVF6,A0955928C2RRWOWZN7UC,0.0,2017-02-17,4.0,0.0000
1,B005WY3TMA,A0955928C2RRWOWZN7UC,0.0,2015-06-14,4.0,0.4404
2,B0090XWU8S,A0955928C2RRWOWZN7UC,0.0,2017-04-22,4.0,0.0000
3,B00FXYTLIK,A0955928C2RRWOWZN7UC,0.0,2015-07-10,4.0,0.7264
4,B00HMZG3YS,A0955928C2RRWOWZN7UC,0.0,2015-07-10,4.0,0.6369


In [10]:
df['rating'] = df['rating'].astype(int)
df['votes'] = df['votes'].astype(int)

In [11]:
df['timestamp'] = pd.to_datetime(df['timestamp'])
df['year']= df['timestamp'].dt.year

In [12]:
def scorechange(n):
  if n<=0.1 :
    r = 0
  else : 
    r = 1
  return r

In [13]:
def votechange(n):
  if n==0:
    r = 'not voted'
  else :
    r = 'voted'
  return r

In [14]:
df['votes'] = df['votes'].apply(votechange)

In [15]:
#df.rename(columns = {'compound_score' : 'sentiment'}, inplace = True)
df['sentiment'] = df['compound_score'].apply(scorechange)

In [16]:
df

,user_id,item_id,votes,timestamp,rating,compound_score,year,sentiment
0,B00191WVF6,A0955928C2RRWOWZN7UC,not voted,2017-02-17,4,0.0000,2017,0
1,B005WY3TMA,A0955928C2RRWOWZN7UC,not voted,2015-06-14,4,0.4404,2015,1
2,B0090XWU8S,A0955928C2RRWOWZN7UC,not voted,2017-04-22,4,0.0000,2017,0
3,B00FXYTLIK,A0955928C2RRWOWZN7UC,not voted,2015-07-10,4,0.7264,2015,1
4,B00HMZG3YS,A0955928C2RRWOWZN7UC,not voted,2015-07-10,4,0.6369,2015,1
...,...,...,...,...,...,...,...,...
99737,B009AYLDSU,AZZYW4YOE1B6E,voted,2013-12-10,5,0.9798,2013,1
99738,B00E055H5O,AZZYW4YOE1B6E,not voted,2015-05-25,4,0.8402,2015,1
99739,B00E8HGWIK,AZZYW4YOE1B6E,not voted,2013-12-01,5,0.9940,2013,1
99740,B00M58CMTM,AZZYW4YOE1B6E,voted,2014-10-08,5,0.9856,2014,1


In [17]:
from sklearn.utils import shuffle
df = shuffle(df,random_state=52)
cutoff = int(len(df)*0.7)
train_df = df[:cutoff]
test_df = df[cutoff:]

In [18]:
train_df.head()

,user_id,item_id,votes,timestamp,rating,compound_score,year,sentiment
96197,B00CST4LB0,AUKQYNHAWG6MS,not voted,2014-05-28,5,0.8751,2014,1
84081,B00N3RFC4Q,ADQT2DW2ZXIOH,voted,2016-11-16,5,0.5106,2016,1
59587,B00GM0OZ4O,A3910BAMCHK81G,not voted,2014-04-05,5,0.8519,2014,1
99600,B00GPAFHIO,AZUA2OFAQCTB8,not voted,2017-12-30,5,0.4404,2017,1
33445,B0111MRLES,A2AKXUY0TXZWIX,not voted,2017-12-02,5,0.8336,2017,1


In [19]:
# train set에 있는 아이템별 감성점수 평균
mean_sentiment_score = train_df.groupby(['item_id']).mean()['compound_score'].to_frame()
mean_sentiment_score

,compound_score
item_id,
A0955928C2RRWOWZN7UC,0.356578
A100UD67AHFODS,0.885525
A100WO06OQR8BQ,0.403119
A1013Q9SD2KIE1,-0.175571
A1018D502E27E6,0.344614
...,...
AZX2RDN9YXZAE,-0.067240
AZXFS8GCTSQ5R,0.935119
AZXJJM4F1SY3,0.512910


In [20]:
mean_sentiment_score.rename(columns = {'compound_score' : 'sentiment'}, inplace = True)

In [21]:
mean_sentiment_score['sentiment'] = mean_sentiment_score['sentiment'].apply(scorechange)
mean_sentiment_score

,sentiment
item_id,
A0955928C2RRWOWZN7UC,1
A100UD67AHFODS,1
A100WO06OQR8BQ,1
A1013Q9SD2KIE1,0
A1018D502E27E6,1
...,...
AZX2RDN9YXZAE,0
AZXFS8GCTSQ5R,1
AZXJJM4F1SY3,1


In [22]:
# train, test 셋에서 compound 삭제 (사용하지 않을 변수)
train_df = train_df.drop('compound_score',axis=1)
test_df = test_df.drop('compound_score',axis=1)

In [23]:
# test 셋에서 감성점수(sentiment) 삭제
test_df = test_df.drop('sentiment',axis=1)

In [24]:
# test 셋에 train set에 있는 감성점수 붙이기(치팅방지)
test_df = pd.merge(mean_sentiment_score,test_df,how='inner',on='item_id')
test_df.head()

,item_id,sentiment,user_id,votes,timestamp,rating,year
0,A0955928C2RRWOWZN7UC,1,B005WY3TMA,not voted,2015-06-14,4,2015
1,A0955928C2RRWOWZN7UC,1,B0090XWU8S,not voted,2017-04-22,4,2017
2,A0955928C2RRWOWZN7UC,1,B00NIYJL64,not voted,2017-02-17,4,2017
3,A100UD67AHFODS,1,B00HPQ0G88,voted,2014-01-30,5,2014
4,A100UD67AHFODS,1,B007G5NNOW,not voted,2015-04-10,5,2015


In [25]:
# User encoding
user_dict = {}
for i in set(df['user_id']):
    user_dict[i] = len(user_dict)
n_user = len(user_dict)
# Item encoding
item_dict = {}
start_point = n_user
for i in set(df['item_id']):
    item_dict[i] = start_point + len(item_dict)
n_item = len(item_dict)
start_point += n_item


# vote encoding
vote_dict = {}
for i in set(df['votes']):
    vote_dict[i] = start_point + len(vote_dict)
n_vote = len(vote_dict)
start_point += n_vote

#year encoding
year_dict = {}
for i in set(df['year']):
    year_dict[i] = start_point + len(year_dict)
n_year = len(year_dict)
start_point += n_year

# sentiment encoding
sentiment_dict = {}
for i in set(df['sentiment']):
    sentiment_dict[i] = start_point + len(sentiment_dict)
n_sentiment = len(sentiment_dict)
start_point += n_sentiment

In [26]:
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

In [27]:
class FM():
    def __init__(self, N, K, train_data, test_data, train_y, test_y, alpha, beta, iterations=100, tolerance=0.005, l2_reg=True, verbose=True):
        self.K = K          # Number of latent factors
        self.N = N          # Number of x (variables)
        # self.n_cases = len(data)            # N of observations
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.l2_reg = l2_reg
        self.tolerance = tolerance
        self.verbose = verbose
        # w 초기화
        self.w = np.random.normal(scale=1./self.N, size=(self.N))
        # v 초기화
        self.v = np.random.normal(scale=1./self.K, size=(self.N, self.K))
        # Train/Test 분리
        self.train_x = train_data
        self.test_x = test_data
        self.train_y = train_y
        self.test_y = test_y
   
    def test(self):                                     # Training 하면서 RMSE 계산 
        # SGD를 iterations 숫자만큼 수행
        best_RMSE = 10000
        best_iteration = 0
        training_process = []
        best_pred = []
        for i in range(self.iterations):
            rmse1 = self.sgd(self.train_x, self.train_y)        # SGD & Train RMSE 계산
            y_pred, rmse2 = self.test_rmse(self.test_x, self.test_y)    # Test RMSE 계산    
            training_process.append((i, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1, rmse1, rmse2))
            if best_RMSE > rmse2:                       # New best record
                best_RMSE = rmse2
                best_iteration = i
                best_pred = y_pred
            elif (rmse2 - best_RMSE) > self.tolerance:  # RMSE is increasing over tolerance
                break
        print(f'-----fm_test : {best_iteration}, {best_RMSE}')
        return training_process, best_pred, best_RMSE
        
    # w, v 업데이트를 위한 Stochastic gradient descent 
    def sgd(self, x_data, y_data):
        y_pred = []
        for data, y in zip(x_data, y_data):
            x_idx = data[0]
            x_0 = np.array(data[1])     # xi axis=0 [1, 2, 3]
            x_1 = x_0.reshape(-1, 1)    # xi axis=1 [[1], [2], [3]]
    
            # biases
            bias_score = np.sum(self.w[x_idx] * x_0)
            
            # score 계산
            vx = self.v[x_idx] * (x_1)          # v matrix * x
            sum_vx = np.sum(vx, axis=0)         # sigma(vx)
            sum_vx_2 = np.sum(vx * vx, axis=0)  # ( v matrix * x )의 제곱
            latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

            # 예측값 계산
            y_hat = bias_score + latent_score
            y_pred.append(y_hat)
            error = y - y_hat
            # w, v 업데이트
            if self.l2_reg:     # regularization이 있는 경우
                self.w[x_idx] += error * self.alpha * (x_0 - self.beta * self.w[x_idx])
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1) - self.beta * self.v[x_idx])
            else:               # regularization이 없는 경우
                self.w[x_idx] += error * self.alpha * x_0
                self.v[x_idx] += error * self.alpha * ((x_1) * sum(vx) - (vx * x_1))
        return RMSE(y_data, y_pred)
            
    def test_rmse(self, x_data, y_data):
        y_pred = []
        for data , y in zip(x_data, y_data):
            y_hat = self.predict(data[0], data[1])
            y_pred.append(y_hat)
        return np.array(y_pred), RMSE(y_data, y_pred)

    def predict(self, idx, x):
        # idx = self.user_id_index[idx]
        # x = self.item_id_index[x]
        x_0 = np.array(x)
        x_1 = x_0.reshape(-1, 1)

        # biases
        bias_score = np.sum(self.w[idx] * x_0)

        # score 계산
        vx = self.v[idx] * (x_1)
        sum_vx = np.sum(vx, axis=0)
        sum_vx_2 = np.sum(vx * vx, axis=0)
        latent_score = 0.5 * np.sum(np.square(sum_vx) - sum_vx_2)

        # 예측값 계산
        y_hat = bias_score + latent_score
        return y_hat
    
    # def get_one_prediction(self, user_id, item_id) :
    #     return self.predict(self.user_id_index[user_id], self.item_id_index[item_id])

In [28]:
class NEW_MF():
    # Initializing the object
    def __init__(self, ratings, K, alpha, beta, iterations, tolerance=0.005, verbose=True):
        self.R = np.array(ratings)
        # user_id, movie_id를 R의 index와 매칭하기 위한 dictionary 생성
        item_id_index = []
        index_item_id = []
        for i, one_id in enumerate(ratings):
            item_id_index.append([one_id, i])
            index_item_id.append([i, one_id])
        self.item_id_index = dict(item_id_index)
        self.index_item_id = dict(index_item_id)        
        user_id_index = []
        index_user_id = []
        for i, one_id in enumerate(ratings.T):
            user_id_index.append([one_id, i])
            index_user_id.append([i, one_id])
        self.user_id_index = dict(user_id_index)
        self.index_user_id = dict(index_user_id)
        # 다른 변수 초기화
        self.num_users, self.num_items = np.shape(self.R)
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations
        self.tolerance = tolerance
        self.verbose = verbose

    # 테스트 셋을 선정하는 메소드 
    def set_test(self, ratings_test):                           # Setting test set
        test_set = []
        for i in range(len(ratings_test)):                      # Selected ratings
            x = self.user_id_index[ratings_test.iloc[i,0]]      # Getting R indice for the given user_id and movie_id
            y = self.item_id_index[ratings_test.iloc[i,1]]
            z = ratings_test.iloc[i,2]
            test_set.append([x, y, z])
            self.R[x, y] = 0                    # Setting test set ratings to 0
        self.test_set = test_set
        return test_set                         # Return test set

    def test(self):                             # Training 하면서 test set의 정확도를 계산하는 메소드 
        # Initializing user-feature and movie-feature matrix
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initializing the bias terms
        self.b_u = np.zeros(self.num_users)
        self.b_d = np.zeros(self.num_items)
        self.b = np.mean(self.R[self.R.nonzero()])

        # List of training samples
        rows, columns = self.R.nonzero()
        self.samples = [(i,j, self.R[i,j]) for i, j in zip(rows, columns)]

        # Stochastic gradient descent for given number of iterations
        best_RMSE = 10000
        best_iteration = 0
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            rmse1 = self.rmse()
            rmse2 = self.test_rmse()
            training_process.append((i, rmse1, rmse2))
            if self.verbose:
                if (i+1) % 10 == 0:
                    print("Iteration: %d ; Train RMSE = %.6f ; Test RMSE = %.6f" % (i+1, rmse1, rmse2))
            if best_RMSE > rmse2:                      # New best record
                best_RMSE = rmse2
                best_iteration = i
            elif (rmse2 - best_RMSE) > self.tolerance: # RMSE is increasing over tolerance
                break
        print(f'-----mf_test :  {best_iteration}, {best_RMSE}')
        return training_process

    # Stochastic gradient descent to get optimized P and Q matrix
    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.get_prediction(i, j)
            e = (r - prediction)

            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_d[j] += self.alpha * (e - self.beta * self.b_d[j])

            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])

    # Computing mean squared error
    def rmse(self):
        xs, ys = self.R.nonzero()
        self.predictions = []
        self.errors = []
        for x, y in zip(xs, ys):
            prediction = self.get_prediction(x, y)
            self.predictions.append(prediction)
            self.errors.append(self.R[x, y] - prediction)
        self.predictions = np.array(self.predictions)
        self.errors = np.array(self.errors)
        return np.sqrt(np.mean(self.errors**2))

    # Test RMSE 계산하는 method 
    def test_rmse(self):
        error = 0
        for one_set in self.test_set:
            predicted = self.get_prediction(one_set[0], one_set[1])
            error += pow(one_set[2] - predicted, 2)
        return np.sqrt(error/len(self.test_set))

    # Ratings for user i and moive j
    def get_prediction(self, i, j):
        prediction = self.b + self.b_u[i] + self.b_d[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    # Ratings for user_id and moive_id
    def get_one_prediction(self, user_id, movie_id):
        return self.get_prediction(self.user_id_index[user_id], self.item_id_index[movie_id])
    

In [29]:
# Total number of x
num_x = start_point             
num_x

10097

In [30]:
def generate_x(x) :
    # Generate X data
    data = []
    y = []
    w0 = np.mean(x['rating'])
    for i in range(len(x)):
        case = x.iloc[i]
        x_index = []
        x_value = []
        x_index.append(user_dict[case['user_id']])     # User id encoding
        x_value.append(1.)
        x_index.append(item_dict[case['item_id']])    # item id encoding
        x_value.append(1.)
        x_index.append(vote_dict[case['votes']])   # vote encoding
        x_value.append(1.)
        x_index.append(year_dict[case['year']])       # year encoding
        x_value.append(1.)
        x_index.append(sentiment_dict[case['sentiment']]) #sentiment encoding
        x_value.append(1.)

        data.append([x_index, x_value])
        y.append(case['rating'] - w0)
        if (i % 10000) == 0:
            print('Encoding ', i, ' cases...')
    return data, y

In [31]:
train_data, train_y = generate_x(train_df) 
test_data, test_y = generate_x(test_df) 

Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...
Encoding  30000  cases...
Encoding  40000  cases...
Encoding  50000  cases...
Encoding  60000  cases...
Encoding  0  cases...
Encoding  10000  cases...
Encoding  20000  cases...


In [32]:
ratings = df.pivot(index = 'user_id', columns ='item_id', values = 'rating').fillna(0)
mf = NEW_MF(ratings, K=16, alpha=0.0013, beta=0.28, iterations=150, tolerance=0.0001, verbose=True)
test_x = test_df[['user_id','item_id','rating','votes','sentiment','year']]
test_set = mf.set_test(test_x)
result = mf.test()

Iteration: 10 ; Train RMSE = 0.943037 ; Test RMSE = 0.953951
Iteration: 20 ; Train RMSE = 0.911082 ; Test RMSE = 0.936840
Iteration: 30 ; Train RMSE = 0.889201 ; Test RMSE = 0.926487
Iteration: 40 ; Train RMSE = 0.873454 ; Test RMSE = 0.919973
Iteration: 50 ; Train RMSE = 0.861766 ; Test RMSE = 0.915853
Iteration: 60 ; Train RMSE = 0.852884 ; Test RMSE = 0.913257
Iteration: 70 ; Train RMSE = 0.846002 ; Test RMSE = 0.911676
Iteration: 80 ; Train RMSE = 0.840580 ; Test RMSE = 0.910776
Iteration: 90 ; Train RMSE = 0.836245 ; Test RMSE = 0.910319
Iteration: 100 ; Train RMSE = 0.832728 ; Test RMSE = 0.910193
Iteration: 110 ; Train RMSE = 0.829837 ; Test RMSE = 0.910255
-----mf_test :  102, 0.9101900212862462


In [33]:
fm1 = FM(num_x, 260, train_data, test_data, train_y, test_y, alpha=0.0001, beta=0.007, iterations=900, tolerance=0.0005, l2_reg=True, verbose=True)
training_process, result0, rmse = fm1.test()  
print('FM RMSE:' , rmse)

Iteration: 10 ; Train RMSE = 0.962944 ; Test RMSE = 0.981948
Iteration: 20 ; Train RMSE = 0.948112 ; Test RMSE = 0.976187
Iteration: 30 ; Train RMSE = 0.939026 ; Test RMSE = 0.971910
Iteration: 40 ; Train RMSE = 0.927741 ; Test RMSE = 0.966318
Iteration: 50 ; Train RMSE = 0.912350 ; Test RMSE = 0.958659
Iteration: 60 ; Train RMSE = 0.892243 ; Test RMSE = 0.949095
Iteration: 70 ; Train RMSE = 0.868889 ; Test RMSE = 0.939173
Iteration: 80 ; Train RMSE = 0.845227 ; Test RMSE = 0.931130
Iteration: 90 ; Train RMSE = 0.824102 ; Test RMSE = 0.926600
Iteration: 100 ; Train RMSE = 0.806775 ; Test RMSE = 0.925659
-----fm_test : 97, 0.9256053574913726
FM RMSE: 0.9256053574913726


In [34]:
def recommender1(recomm_list, mf):
    recommendations = []
    for i in range(len(recomm_list)):
        recommendations.append(mf.get_one_prediction(recomm_list[i,0], recomm_list[i,1]))
    return np.array(recommendations)

# RMSE 계산을 위한 함수
def RMSE2(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

recomm_list = np.array(test_x.loc[:, ['user_id', 'item_id']])        # Data for testing context-DL model
result1 = recommender1(recomm_list, mf)

In [35]:
weight_rmse=[]
i = 0.72
weight = [i, 1-i]
predictions = []
w0 = np.mean(df['rating'])
for i, number in enumerate(result0):
    predictions.append(result0[i] * weight[0] + (result1[i]- w0) * weight[1])
weight_rmse.append([weight[0], round(weight[1],2), round(RMSE2(test_y, predictions),7)])
print("Weights - %.2f : %.2f ; RMSE = %.7f" % (weight[0], weight[1], RMSE2(test_y, predictions)))

Weights - 0.72 : 0.28 ; RMSE = 0.9156467


#########연습

In [ ]:
# 가중치 별 테스트
for i in [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]:
    weight = [i, 1-i]
    predictions = []
    w0 = np.mean(df['rating'])
    for i, number in enumerate(result0):
        predictions.append(result0[i] * weight[0] + (result1[i]-w0) * weight[1])
    print("Weights - %.2f : %.2f ; RMSE = %.7f" % (weight[0], weight[1], RMSE2(test_y, predictions)))

Weights - 0.10 : 0.90 ; RMSE = 0.9092892
Weights - 0.20 : 0.80 ; RMSE = 0.9088267
Weights - 0.30 : 0.70 ; RMSE = 0.9089360
Weights - 0.40 : 0.60 ; RMSE = 0.9096168
Weights - 0.50 : 0.50 ; RMSE = 0.9108678
Weights - 0.60 : 0.40 ; RMSE = 0.9126867
Weights - 0.70 : 0.30 ; RMSE = 0.9150702
Weights - 0.80 : 0.20 ; RMSE = 0.9180138
Weights - 0.90 : 0.10 ; RMSE = 0.9215121


In [ ]:
# 가중치 별 테스트
for i in [0.71, 0.72, 0.73, 0.74, 0.75, 0.76, 0.77, 0.78, 0.79]:
    weight = [1-i, i]
    predictions = []
    w0 = np.mean(df['rating'])
    for i, number in enumerate(result0):
        predictions.append(result0[i] * weight[0] + (result1[i]-w0) * weight[1])
    print("Weights - %.2f : %.2f ; RMSE = %.7f" % (weight[0], weight[1], RMSE2(test_y, predictions)))

Weights - 0.29 : 0.71 ; RMSE = 0.9088993
Weights - 0.28 : 0.72 ; RMSE = 0.9088684
Weights - 0.27 : 0.73 ; RMSE = 0.9088432
Weights - 0.26 : 0.74 ; RMSE = 0.9088237
Weights - 0.25 : 0.75 ; RMSE = 0.9088099
Weights - 0.24 : 0.76 ; RMSE = 0.9088018
Weights - 0.23 : 0.77 ; RMSE = 0.9087995
Weights - 0.22 : 0.78 ; RMSE = 0.9088028
Weights - 0.21 : 0.79 ; RMSE = 0.9088119


In [ ]:
r12=[0.9193168, 
0.9193131,
0.9193154,
0.9193234,
0.9193373,
0.9193569,
0.9193825,
0.9194138,
0.9194510]
r32=[0.9238420,
0.9238413,
0.9238454,
0.9238544,
0.9238683,
0.9238870,
0.9239106,
0.9239391,
0.9239724]
r52=[0.9088993,
0.9088684,
0.9088432,
0.9088237,
0.9088099,
0.9088018,
0.9087995,
0.9088028,
0.9088119]
for i in range(len(r12)):
  print((r12[i]+r32[i]+r52[i])/3)

0.9173527
0.9173409333333332
0.9173346666666667
0.9173338333333333
0.9173385000000001
0.9173485666666666
0.9173642000000001
0.9173852333333333
0.9174117666666667
